1. Project Title & Team Information

Title:  Accident Risk Analysis: Urban vs. Rural Populations

Team Members: Dave Woodford, Tim Mai, and Dave Woodford

#TO DO: Team Members Table (Name, Role, Skills, Contribution)

2. Project Definition & Motivation

Objective: "Does accident frequency increase in high-population counties? Do factors differ between rural and metro areas?"

Stakeholders: Dept of Transporation, Railroad Administration, Railroad companies, local municipaliteis, and the public

Potential Application: A risk-assessment tool or "safe travel" dashboard


3. Data Sources & Selection

Source A: Rail Equipment Accident Data (Form 54).
    Justification: Primary source for accident frequency and cause codes.
    Link: https://data.transportation.gov/Railroads/Rail-Equipment-Accident-Incident-Data-Form-54-/85tf-25kj/about_data

Source B: USDA Rural-Urban Continuum Codes (2023).
    Justification: Provides the necessary demographic context (Rural vs. Metro) to normalize accident data.
    Link: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes

#CODE TO INCLUDE: import libraries

#CODE TO INCLUDE: load data

4. Methodlogy: Data Cleaning

- Explain: "We padded State (2 digits) and County (3 digits) to create a valid 5-digit FIPS key for merging."

In [ ]:
# CODE TO INCLUDE: FIPS code repair logic
# df_accident["FIPS"] = ...

5. Methodology: The Merge

In [ ]:
# CODE TO INCLUDE: left join on FIPS codes

# CODE TO INCLUDE: validation of successful merge
#df.shape or df.head()

6. Initial Exploratory Analysis (EDA)

The Question: "Is there a correlation between population size and raw accident count?"

The Visualization: Your scatter plot (plt.scatter(x,y)).

The Caption/Interpretation: "The plot shows a positive correlation (0.58), but outliers exist. This suggests that while population is a factor, other variables (track density, traffic volume) likely play a role."

7. Limitations and Future Improvements

Current Limitations: The analysis currently looks at raw counts, not normalized by "train miles" or "track miles" (traffic density).

Missing Variables: Weather, Time of Day, Seasonality, etc. 

8. Dissemination & Next Steps

Target Audience: 
    US Population (safety consciousness)
    Gov Agencies (maintenance and policy prioritization)
    Railroad companies

Next Steps:
    Analyzing specific "Cause Codes" (Human error vs. Equipment failure) across those bins.